In [220]:
import numpy as np
import pandas as pd
import wave
import librosa
import re
from allennlp.commands.elmo import ElmoEmbedder
# from bert_serving.client import BertClient

prefix = '/Users/apple/Downloads/depression/'

elmo = ElmoEmbedder()
# bc = BertClient(ip='100.66.165.12')

train_split_df = pd.read_csv(prefix+'train_split_Depression_AVEC2017 (1).csv')
test_split_df = pd.read_csv(prefix+'dev_split_Depression_AVEC2017.csv')
train_split_num = train_split_df[['Participant_ID']]['Participant_ID'].tolist()
test_split_num = test_split_df[['Participant_ID']]['Participant_ID'].tolist()
# train_split_clabel = train_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
# test_split_clabel = test_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
train_split_rlabel = train_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()
test_split_rlabel = test_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()

topics = []
with open('/Users/apple/Downloads/depression/queries.txt', 'r') as f:
    for line in f.readlines():
        topics.append(line.strip('\n').strip())
        

def identify_topics(sentence):
    if sentence in topics:
        return True
    return False

def extract_features(number, text_features, target, mode, text_targets):
    
    transcript = pd.read_csv(prefix+'{0}_P/{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')
    
    
    time_range = []
    responses = []
    response = ''
    response_flag = False
    start_time = 0
    stop_time = 0

    signal = []
    
    global counter1, counter2

    for t in transcript.itertuples():
        # participant一句话结束
        if getattr(t,'speaker') == 'Ellie':
#             if '(' in getattr(t,'value'):
#                 content = re.findall(re.compile(r'[(](.*?)[)]', re.S), getattr(t,'value'))[0]
#                 print(content)
#             else:
#                 content = getattr(t,'value').strip()
            content = getattr(t,'value').strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    responses.append(response.strip())
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    responses.append(response)
                response = ''
        elif getattr(t,'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t,'value'):
                continue
            elif response_flag:
                content = getattr(t,'value').split('\n')[0].strip()
#                 if '<' in getattr(t,'value'):
#                     content = re.sub(u"\\<.*?\\>", "", content)
                response+=' '+content
        
    text_feature = elmo.embed_sentence(responses).mean(0)
#     text_feature = bc.encode(responses)
#     while text_feature.shape[0] < 30:
#         print(number)
#         text_feature = np.vstack((text_feature, np.zeros(text_feature.shape[1])))
    print(text_feature.shape)
#     text_features.append(text_feature[:30])
    text_features.append(text_feature)
    text_targets.append(target)
    
def extract_features1(number, text_features, target, mode, text_targets):
    
    transcript = pd.read_csv(prefix+'{0}_P/{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')
    
    
    time_range = []
    responses = []
    response = ''
    response_flag = False
    start_time = 0
    stop_time = 0

    signal = []
    
    global counter1, counter2

    for t in transcript.itertuples():
        # participant一句话结束
        if getattr(t,'speaker') == 'Ellie':
            if '(' in getattr(t,'value'):
                content = re.findall(re.compile(r'[(](.*?)[)]', re.S), getattr(t,'value'))[0]
            else:
                content = getattr(t,'value').strip()
            content = getattr(t,'value').strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    responses.append(response.strip())
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    responses.append(response)
                response = ''
        elif getattr(t,'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t,'value'):
                continue
            elif response_flag:
                response+=' '+getattr(t,'value').split('\n')[0].strip()
        
    text_feature = elmo.embed_sentence(responses).mean(0)
    print(text_feature.shape)
    text_features.append(text_feature)
    if target == 1:
        counter1 += len(text_feature)
    else:
        counter2 += len(text_feature)
    text_targets.append(target)

counter1 = 0
counter2 = 0
    
# training set
text_features_train = []
text_ctargets_train = []

# test set
text_features_test = []
text_ctargets_test = []

# ======================= classification =======================

# training set
# for index in range(len(train_split_num)):
#     extract_features(train_split_num[index], text_features_train, train_split_clabel[index], 'train', text_ctargets_train)
    
# # test set
# for index in range(len(test_split_num)):
#     extract_features(test_split_num[index], text_features_test, test_split_clabel[index], 'test', text_ctargets_test)

# ======================= classification =======================

# training set
for index in range(len(train_split_num)):
    extract_features(train_split_num[index], text_features_train, train_split_rlabel[index], 'train', text_ctargets_train)
    
# test set
for index in range(len(test_split_num)):
    extract_features(test_split_num[index], text_features_test, test_split_rlabel[index], 'test', text_ctargets_test)
print(np.shape(text_features_train), np.shape(text_features_test), np.shape(text_ctargets_train), np.shape(text_ctargets_test))


(36, 1024)
(49, 1024)
(46, 1024)
(47, 1024)
(42, 1024)
(43, 1024)
(48, 1024)
(55, 1024)
(50, 1024)
(38, 1024)
(44, 1024)
(63, 1024)
(59, 1024)
(54, 1024)
(44, 1024)
(38, 1024)
(56, 1024)
(55, 1024)
(41, 1024)
(48, 1024)
(46, 1024)
(56, 1024)
(41, 1024)
(49, 1024)
(48, 1024)
(47, 1024)
(63, 1024)
(55, 1024)
(37, 1024)
(50, 1024)
(56, 1024)
(44, 1024)
(51, 1024)
(32, 1024)
(43, 1024)
(38, 1024)
(37, 1024)
(30, 1024)
(41, 1024)
(33, 1024)
(42, 1024)
(33, 1024)
(42, 1024)
(34, 1024)
(28, 1024)
(31, 1024)
(25, 1024)
(54, 1024)
(43, 1024)
(39, 1024)
(46, 1024)
(40, 1024)
(35, 1024)
(38, 1024)
(34, 1024)
(48, 1024)
(42, 1024)
(47, 1024)
(43, 1024)
(39, 1024)
(54, 1024)
(53, 1024)
(43, 1024)
(41, 1024)
(29, 1024)
(53, 1024)
(35, 1024)
(44, 1024)
(51, 1024)
(37, 1024)
(51, 1024)
(40, 1024)
(48, 1024)
(47, 1024)
(48, 1024)
(51, 1024)
(46, 1024)
(47, 1024)
(37, 1024)
(44, 1024)
(68, 1024)
(52, 1024)
(37, 1024)
(43, 1024)
(34, 1024)
(38, 1024)
(32, 1024)
(33, 1024)
(64, 1024)
(34, 1024)
(48, 1024)

In [ ]:
# print("Saving npz file locally...")

# np.savez(prefix+'data/text/train_samples.npz', text_features_train)
# np.savez(prefix+'data/text/train_labels.npz', text_features_test)
# np.savez(prefix+'data/text/test_samples.npz', text_ctargets_train)
# np.savez(prefix+'data/text/test_labels.npz', text_ctargets_test)

prefix = '/Users/apple/Downloads/depression/'

text_features_train = np.load(prefix+'data/text/train_samples.npz')['arr_0']
text_features_test = np.load(prefix+'data/text/train_labels.npz')['arr_0']
text_ctargets_train = np.load(prefix+'data/text/test_samples.npz')['arr_0']
text_ctargets_test = np.load(prefix+'data/text/test_labels.npz')['arr_0']

In [221]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.optim as optim
from sklearn.metrics import confusion_matrix

In [222]:
class BiLSTM(nn.Module):
    
    def __init__(self, config):
        super(BiLSTM, self).__init__()
        self.num_classes = config['num_classes']
        self.learning_rate = config['learning_rate']
        self.dropout = config['dropout']
        self.hidden_dims = config['hidden_dims']
        self.rnn_layers = config['rnn_layers']
        self.embedding_size = config['embedding_size']
        self.bidirectional = config['bidirectional']

        self.build_model()
        self.init_weight()
        
    def init_weight(net):
        for name, param in net.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_uniform_(param)

    def build_model(self):
        # attention layer
        self.attention_layer = nn.Sequential(
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(inplace=True)
        )
        # self.attention_weights = self.attention_weights.view(self.hidden_dims, 1)

        # 双层lstm
        self.lstm_net = nn.LSTM(self.embedding_size, self.hidden_dims,
                                num_layers=self.rnn_layers, dropout=self.dropout,
                                bidirectional=self.bidirectional)
        
#         self.init_weight()
        
        # FC层
#         self.fc_out = nn.Linear(self.hidden_dims, self.num_classes)
        self.fc_out = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(inplace=True),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.num_classes),
            nn.ReLU(),
        )

    def attention_net_with_w(self, lstm_out, lstm_hidden):
        '''
        :param lstm_out:    [batch_size, len_seq, n_hidden * 2]
        :param lstm_hidden: [batch_size, num_layers * num_directions, n_hidden]
        :return: [batch_size, n_hidden]
        '''
        lstm_tmp_out = torch.chunk(lstm_out, 2, -1)
        # h [batch_size, time_step, hidden_dims]
        h = lstm_tmp_out[0] + lstm_tmp_out[1]
#         h = lstm_out
        # [batch_size, num_layers * num_directions, n_hidden]
        lstm_hidden = torch.sum(lstm_hidden, dim=1)
        # [batch_size, 1, n_hidden]
        lstm_hidden = lstm_hidden.unsqueeze(1)
        # atten_w [batch_size, 1, hidden_dims]
        atten_w = self.attention_layer(lstm_hidden)
        # m [batch_size, time_step, hidden_dims]
        m = nn.Tanh()(h)
        # atten_context [batch_size, 1, time_step]
        atten_context = torch.bmm(atten_w, m.transpose(1, 2))
        # softmax_w [batch_size, 1, time_step]
        softmax_w = F.softmax(atten_context, dim=-1)
        # context [batch_size, 1, hidden_dims]
        context = torch.bmm(softmax_w, h)
        result = context.squeeze(1)
        return result

    def forward(self, x):
        
        # x : [len_seq, batch_size, embedding_dim]
        x = x.permute(1, 0, 2)
        output, (final_hidden_state, final_cell_state) = self.lstm_net(x)
        # output : [batch_size, len_seq, n_hidden * 2]
        output = output.permute(1, 0, 2)
        # final_hidden_state : [batch_size, num_layers * num_directions, n_hidden]
        final_hidden_state = final_hidden_state.permute(1, 0, 2)
        # final_hidden_state = torch.mean(final_hidden_state, dim=0, keepdim=True)
        # atten_out = self.attention_net(output, final_hidden_state)
        atten_out = self.attention_net_with_w(output, final_hidden_state)
        return self.fc_out(atten_out)
    
        

In [284]:
# data imbalance
X_train = []
Y_train = []
X_test = []
Y_test = []

counter = 0

cut = 10
debt = 0

for i in range(len(text_features_train)):
#     if text_ctargets_train[i] == 1:
    if text_ctargets_train[i] >= 10:
        times = 3+debt if counter < 46 else 2+debt
#         print(times, text_features_train[i].shape, debt)
        for j in range(times):
            if (j+1)*cut > len(text_features_train[i]):
                debt+=1
                continue
            X_train.append(text_features_train[i][j*cut:(j+1)*cut])
            Y_train.append(text_ctargets_train[i])
            if debt > 0:
                debt -= 1
            counter+=1
    else:
        X_train.append(text_features_train[i][:cut])
        Y_train.append(text_ctargets_train[i])
        
        
for i in range(len(text_features_test)):
    X_test.append(text_features_test[i][:cut])
    Y_test.append(text_ctargets_test[i])

# for i in range(len(text_features_train)):
#     if text_ctargets_train[i] == 1:
#         times = int(len(text_features_train[i]) / 10)
#         for j in range(times):
#             X_train.append(text_features_train[i][j*10:(j+1)*10])
#             Y_train.append(text_ctargets_train[i])
#             counter+=1
#     else:
#         times = 
#         X_train.append(text_features_train[i][:10])
#         Y_train.append(text_ctargets_train[i])
        
        
# for i in range(len(text_features_test)):
#     X_test.append(text_features_test[i][:10])
#     Y_test.append(text_ctargets_test[i])
    
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [285]:
X_train.shape

(154, 10, 1024)

In [340]:

config = {
    'num_classes': 1,
    'dropout': 0.5,
    'rnn_layers': 2,
    'embedding_size': 1024,
    'batch_size': 8,
    'epochs': 200,
    'learning_rate': 5e-4,
    'hidden_dims': 128,
    'bidirectional': True
}

model = BiLSTM(config)

# if args.cuda:
#     model = model.cuda()
#     X_train = X_train.cuda()
#     Y_train = Y_train.cuda()
#     X_test = X_test.cuda()
#     Y_test = Y_test.cuda()

optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
# optimizer = optim.Adam(model.parameters())
# criterion = nn.CrossEntropyLoss()
criterion = nn.SmoothL1Loss()
max_f1 = -1
max_acc = -1
train_acc = -1
min_mae = 100

def save(model, filename):
    save_filename = '{}.pt'.format(filename)
    torch.save(model, save_filename)
    print('Saved as %s' % save_filename)
    
def standard_confusion_matrix(y_test, y_test_pred):
    """
    Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_test, y_test_pred)
    return np.array([[tp, fp], [fn, tn]])

def model_performance(y_test, y_test_pred_proba):
    """
    Evaluation metrics for network performance.
    """
    y_test_pred = y_test_pred_proba.data.max(1, keepdim=True)[1]

    # Computing confusion matrix for test dataset
    conf_matrix = standard_confusion_matrix(y_test, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    return y_test_pred, conf_matrix

def plot_roc_curve(y_test, y_score):
    """
    Plots ROC curve for final trained model. Code taken from:
    https://vkolachalama.blogspot.com/2016/05/keras-implementation-of-mlp-neural.html
    """
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.savefig(prefix+'images/BiLSTM_roc.png')
    plt.close()


def train(epoch):
    global lr, train_acc
    model.train()
    batch_idx = 1
    total_loss = 0
    correct = 0
    for i in range(0, X_train.shape[0], config['batch_size']):
        if i + config['batch_size'] > X_train.shape[0]:
            x, y = X_train[i:], Y_train[i:]
        else:
            x, y = X_train[i:(i+config['batch_size'])], Y_train[i:(i+config['batch_size'])]
        if False:
            x, y = Variable(torch.from_numpy(x).type(torch.FloatTensor), requires_grad=True).cuda(), Variable(torch.from_numpy(y)).cuda()
        else:
#             x, y = Variable(torch.from_numpy(x).type(torch.FloatTensor), requires_grad=True), Variable(torch.from_numpy(y))
            x, y = Variable(torch.from_numpy(x).type(torch.FloatTensor), requires_grad=True), Variable(torch.from_numpy(y)).type(torch.FloatTensor)
        # 将模型的参数梯度设置为0
        optimizer.zero_grad()
        output = model(x)
#         pred = output.data.max(1, keepdim=True)[1]
#         correct += pred.eq(y.data.view_as(pred)).cpu().sum()
        loss = criterion(output.flatten(), y)
        # 后向传播调整参数
        loss.backward()
        # 根据梯度更新网络参数
        optimizer.step()
        batch_idx += 1
        # loss.item()能够得到张量中的元素值
        total_loss += loss.item()
        
#     train_acc = correct
    train_acc = total_loss/batch_idx
    cur_loss = total_loss
#     print('Train Epoch: {:2d}\t Learning rate: {:.4f}\tLoss: {:.6f}\t Accuracy: {}/{} ({:.0f}%)\n '.format(
#                 epoch+1, config['learning_rate'], cur_loss, correct, len(X_train),
#         100. * correct / len(X_train)))
    print('Train Epoch: {:2d}\t Learning rate: {:.4f}\t Loss: {:.6f}\t '.format(
                epoch+1, config['learning_rate'], total_loss/batch_idx))


def evaluate(model):
    model.eval()
    batch_idx = 1
    total_loss = 0
    global max_f1, max_acc, min_mae
    with torch.no_grad():
        x, y = Variable(torch.from_numpy(X_test).type(torch.FloatTensor), requires_grad=True), Variable(torch.from_numpy(Y_test)).type(torch.FloatTensor)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output.flatten(), y)
        total_loss += loss.item()
#         print(y, output)
#         y_test_pred, conf_matrix = model_performance(y, output)
        print('\nTest set: Average loss: {:.4f} \t MAE: {:.4f}\n'.format(total_loss, F.l1_loss(output.flatten(), y)))
        
#         # custom evaluation metrics
#         print('Calculating additional test metrics...')
#         accuracy = float(conf_matrix[0][0] + conf_matrix[1][1]) / np.sum(conf_matrix)
#         precision = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[0][1])
#         recall = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[1][0])
#         f1_score = 2 * (precision * recall) / (precision + recall)
#         print("Accuracy: {}".format(accuracy))
#         print("Precision: {}".format(precision))
#         print("Recall: {}".format(recall))
#         print("F1-Score: {}\n".format(f1_score))
#         print('='*89)
        
#         if max_f1 <= f1_score and train_acc > 151:
#             max_f1 = f1_score
#             max_acc = accuracy
#             save(model, 'BiLSTM_elmo_{}_{:.2f}'.format(config['hidden_dims'], max_f1))          
#             print('*'*64)
#             print('model saved: f1: {}\tacc: {}'.format(max_f1, max_acc))
#             print('*'*64)
        if min_mae >= F.l1_loss(output.flatten(), y) and train_acc < 2.0:
            min_mae = F.l1_loss(output.flatten(), y)
            save(model, 'BiLSTM_reg_{}_{:.2f}'.format(config['hidden_dims'], min_mae))          
            print('*'*64)
            print('model saved: f1: {}\tacc: {}'.format(min_mae, train_acc))
            print('*'*64)

    return total_loss


In [341]:
for ep in range(1, config['epochs']):
    train(ep)
    tloss = evaluate(model)

Train Epoch:  2	 Learning rate: 0.0005	 Loss: 6.222528	 

Test set: Average loss: 5.2454 	 MAE: 5.7352

Train Epoch:  3	 Learning rate: 0.0005	 Loss: 4.801173	 

Test set: Average loss: 5.5409 	 MAE: 6.0273

Train Epoch:  4	 Learning rate: 0.0005	 Loss: 4.431237	 

Test set: Average loss: 5.2539 	 MAE: 5.7438

Train Epoch:  5	 Learning rate: 0.0005	 Loss: 4.382258	 

Test set: Average loss: 5.3497 	 MAE: 5.8280

Train Epoch:  6	 Learning rate: 0.0005	 Loss: 4.518691	 

Test set: Average loss: 5.4297 	 MAE: 5.9101

Train Epoch:  7	 Learning rate: 0.0005	 Loss: 4.425840	 

Test set: Average loss: 5.3102 	 MAE: 5.7929

Train Epoch:  8	 Learning rate: 0.0005	 Loss: 4.562018	 

Test set: Average loss: 5.0805 	 MAE: 5.5750

Train Epoch:  9	 Learning rate: 0.0005	 Loss: 4.654327	 

Test set: Average loss: 5.2846 	 MAE: 5.7715

Train Epoch: 10	 Learning rate: 0.0005	 Loss: 4.565650	 

Test set: Average loss: 5.2604 	 MAE: 5.7470

Train Epoch: 11	 Learning rate: 0.0005	 Loss: 4.451685	 

Test s

Train Epoch: 78	 Learning rate: 0.0005	 Loss: 1.062055	 

Test set: Average loss: 3.9293 	 MAE: 4.4046

Train Epoch: 79	 Learning rate: 0.0005	 Loss: 1.161488	 

Test set: Average loss: 4.0311 	 MAE: 4.4819

Train Epoch: 80	 Learning rate: 0.0005	 Loss: 1.155397	 

Test set: Average loss: 4.4762 	 MAE: 4.9510

Train Epoch: 81	 Learning rate: 0.0005	 Loss: 1.124911	 

Test set: Average loss: 3.9555 	 MAE: 4.4218

Train Epoch: 82	 Learning rate: 0.0005	 Loss: 1.087112	 

Test set: Average loss: 3.9758 	 MAE: 4.4648

Train Epoch: 83	 Learning rate: 0.0005	 Loss: 1.143451	 

Test set: Average loss: 4.1492 	 MAE: 4.6386

Train Epoch: 84	 Learning rate: 0.0005	 Loss: 1.021506	 

Test set: Average loss: 4.2033 	 MAE: 4.6706

Train Epoch: 85	 Learning rate: 0.0005	 Loss: 1.397525	 

Test set: Average loss: 4.3981 	 MAE: 4.8657

Train Epoch: 86	 Learning rate: 0.0005	 Loss: 0.968711	 

Test set: Average loss: 3.7188 	 MAE: 4.2074

Train Epoch: 87	 Learning rate: 0.0005	 Loss: 1.095073	 

Test s

Train Epoch: 157	 Learning rate: 0.0005	 Loss: 0.992670	 

Test set: Average loss: 4.0878 	 MAE: 4.5467

Train Epoch: 158	 Learning rate: 0.0005	 Loss: 0.865013	 

Test set: Average loss: 4.3303 	 MAE: 4.7961

Train Epoch: 159	 Learning rate: 0.0005	 Loss: 0.814098	 

Test set: Average loss: 4.0651 	 MAE: 4.5495

Train Epoch: 160	 Learning rate: 0.0005	 Loss: 0.843722	 

Test set: Average loss: 3.9392 	 MAE: 4.4237

Train Epoch: 161	 Learning rate: 0.0005	 Loss: 0.778094	 

Test set: Average loss: 4.1093 	 MAE: 4.5677

Train Epoch: 162	 Learning rate: 0.0005	 Loss: 0.760861	 

Test set: Average loss: 4.0424 	 MAE: 4.4901

Train Epoch: 163	 Learning rate: 0.0005	 Loss: 0.976117	 

Test set: Average loss: 3.9120 	 MAE: 4.3788

Train Epoch: 164	 Learning rate: 0.0005	 Loss: 0.747449	 

Test set: Average loss: 4.1145 	 MAE: 4.5898

Train Epoch: 165	 Learning rate: 0.0005	 Loss: 0.960359	 

Test set: Average loss: 4.3442 	 MAE: 4.8045

Train Epoch: 166	 Learning rate: 0.0005	 Loss: 0.837411

In [9]:
lstm_model = torch.load('/Users/apple/Downloads/depression/BiLSTM_elmo_128_0.83.pt')
model = BiLSTM(config)
model.load_state_dict(lstm_model.state_dict())
evaluate(model)

Confusion Matrix:
[[10  2]
 [ 2 21]]

Test set: Average loss: 1.1024
Calculating additional test metrics...
Accuracy: 0.8857142857142857
Precision: 0.8333333333333334
Recall: 0.8333333333333334
F1-Score: 0.8333333333333334



1.102392554283142

In [342]:
lstm_model = torch.load('/Users/apple/Downloads/depression/BiLSTM_reg_128_3.88.pt')
model = BiLSTM(config)
model.load_state_dict(lstm_model.state_dict())
evaluate(model)


Test set: Average loss: 3.4328 	 MAE: 3.8846

Saved as BiLSTM_reg_128_3.88.pt
****************************************************************
model saved: f1: 3.8845977783203125	acc: 0.8374860261877378
****************************************************************


3.432814359664917